- Author: https://github.com/muz-rdm

## Executive Summary

### Objective
- To ensure a good quality of data by pre-processing the raw data before being consumed in analytics.

### Summary
- Data quality was checked.
- Data wrangling was conducted, acceptable quality of datasets were confirmed for usage in EDA & ML model development.
- Cleansed dataset was save into a new csv.

## Import Library

In [1]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter("ignore")
pd.set_option('display.max_columns', None)

## Load Data

In [2]:
# Load dataset from csv file
df = pd.read_csv('../data/hotel_bookings.csv')
df.shape

(119390, 32)

## Process Data

- Process null columns 
- Process duplicated rows
- Process outliers
- Derive new columns 
- Save cleansed data as new CSV

### Process Null Columns

In [3]:
# Check null
df.isna().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [4]:
# Check columns with null
features = ['children', 'country', 'agent', 'company']

for feat in features:
    perc = len(df[df[feat].isna()])/len(df)*100
    perc = round(perc, 1)
    print(f'Null in {feat}:', perc, '%')
    print(df[feat].describe(), '\n')

Null in children: 0.0 %
count    119386.000000
mean          0.103890
std           0.398561
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          10.000000
Name: children, dtype: float64 

Null in country: 0.4 %
count     118902
unique       177
top          PRT
freq       48590
Name: country, dtype: object 

Null in agent: 13.7 %
count    103050.000000
mean         86.693382
std         110.774548
min           1.000000
25%           9.000000
50%          14.000000
75%         229.000000
max         535.000000
Name: agent, dtype: float64 

Null in company: 94.3 %
count    6797.000000
mean      189.266735
std       131.655015
min         6.000000
25%        62.000000
50%       179.000000
75%       270.000000
max       543.000000
Name: company, dtype: float64 



In [5]:
# For 'children', impute the null with median value
if df['children'].notna().any():
    mode = df['children'].mode()[0]
    df['children'].fillna(value=mode, inplace=True)
    
# For 'country', impute the null with most frequent value
if df['country'].notna().any():
    mode = df['country'].mode()[0]
    df['country'].fillna(value=mode, inplace=True)

# For 'agent', impute the null with most frequent value
if df['agent'].notna().any():
    mode = df['agent'].mode()[0]
    df['agent'].fillna(value=mode, inplace=True)
    
# For 'company', drop the column due to large number of null
df.drop('company', axis=1, inplace=True)

### Process Duplicated Rows

In [6]:
# Check duplicated data
df[df.duplicated()] 
# -> Since there is no unique reservations ID and the duplicated number of rows is significant, keep the data.

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
5,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,0,Transient,98.00,0,1,Check-Out,2015-07-03
22,Resort Hotel,0,72,2015,July,27,1,2,4,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,A,A,1,No Deposit,250.0,0,Transient,84.67,0,1,Check-Out,2015-07-07
43,Resort Hotel,0,70,2015,July,27,2,2,3,2,0.0,0,HB,ROU,Direct,Direct,0,0,0,E,E,0,No Deposit,250.0,0,Transient,137.00,0,1,Check-Out,2015-07-07
138,Resort Hotel,1,5,2015,July,28,5,1,0,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,240.0,0,Transient,97.00,0,0,Canceled,2015-07-01
200,Resort Hotel,0,0,2015,July,28,7,0,1,1,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,0,Transient,109.80,0,3,Check-Out,2015-07-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119349,City Hotel,0,186,2017,August,35,31,0,3,2,0.0,0,BB,DEU,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.0,0,Transient,126.00,0,2,Check-Out,2017-09-03
119352,City Hotel,0,63,2017,August,35,31,0,3,3,0.0,0,BB,SWE,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.0,0,Transient-Party,195.33,0,2,Check-Out,2017-09-03
119353,City Hotel,0,63,2017,August,35,31,0,3,3,0.0,0,BB,SWE,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.0,0,Transient-Party,195.33,0,2,Check-Out,2017-09-03
119354,City Hotel,0,63,2017,August,35,31,0,3,3,0.0,0,BB,SWE,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.0,0,Transient-Party,195.33,0,2,Check-Out,2017-09-03


### Process Outliers

In [7]:
# Check outliers in original data
df_ori = pd.DataFrame(df.describe(include='all'))
df_ori

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
count,119390,119390.000000,119390.000000,119390.000000,119390,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390,119390,119390,119390,119390.000000,119390.000000,119390.000000,119390,119390,119390.000000,119390,119390.000000,119390.000000,119390,119390.000000,119390.000000,119390.000000,119390,119390
unique,2,NaN,NaN,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,177,8,5,NaN,NaN,NaN,10,12,NaN,3,NaN,NaN,4,NaN,NaN,NaN,3,926
top,City Hotel,NaN,NaN,NaN,August,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BB,PRT,Online TA,TA/TO,NaN,NaN,NaN,A,A,NaN,No Deposit,NaN,NaN,Transient,NaN,NaN,NaN,Check-Out,2015-10-21
freq,79330,NaN,NaN,NaN,13877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92310,49078,56477,97870,NaN,NaN,NaN,85994,74053,NaN,104641,NaN,NaN,89613,NaN,NaN,NaN,75166,1461
mean,NaN,0.370416,104.011416,2016.156554,NaN,27.165173,15.798241,0.927599,2.500302,1.856403,0.103886,0.007949,NaN,NaN,NaN,NaN,0.031912,0.087118,0.137097,NaN,NaN,0.221124,NaN,76.060080,2.321149,NaN,101.831122,0.062518,0.571363,NaN,NaN
std,NaN,0.482918,106.863097,0.707476,NaN,13.605138,8.780829,0.998613,1.908286,0.579261,0.398555,0.097436,NaN,NaN,NaN,NaN,0.175767,0.844336,1.497437,NaN,NaN,0.652306,NaN,106.323205,17.594721,NaN,50.535790,0.245291,0.792798,NaN,NaN
min,NaN,0.000000,0.000000,2015.000000,NaN,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,0.000000,NaN,1.000000,0.000000,NaN,-6.380000,0.000000,0.000000,NaN,NaN
25%,NaN,0.000000,18.000000,2016.000000,NaN,16.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,0.000000,NaN,9.000000,0.000000,NaN,69.290000,0.000000,0.000000,NaN,NaN
50%,NaN,0.000000,69.000000,2016.000000,NaN,28.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,0.000000,NaN,9.000000,0.000000,NaN,94.575000,0.000000,0.000000,NaN,NaN
75%,NaN,1.000000,160.000000,2017.000000,NaN,38.000000,23.000000,2.000000,3.000000,2.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,0.000000,NaN,152.000000,0.000000,NaN,126.000000,0.000000,1.000000,NaN,NaN


In [8]:
# Process outliers in original data

# Impute outlier value with 0
df.loc[df['adults']>4, 'adults'] = 0
df.loc[df['children']>4, 'children'] = 0
df.loc[df['babies']>4, 'babies'] = 0

# 'Meal' contains values "Undefined", which is equal to SC
df['meal'].replace('Undefined', 'SC', inplace=True)

### Derive New Columns

In [9]:
# Derive new columns

# Create 'total_stay_nights'
df['total_stay_nights'] = df['stays_in_week_nights'] + df['stays_in_weekend_nights']

# Create 'kids'& 'num_pax'
df['kids'] = df['children'] + df['babies'] 
df['num_pax'] = df['adults'] + df['kids'] 

In [10]:
# Check outliers in derived data
df_der = pd.DataFrame(df[['total_stay_nights', 'num_pax']].describe())
df_der

,total_stay_nights,num_pax
count,119390.000000,119390.000000
mean,3.427900,1.964687
std,2.557439,0.651295
min,0.000000,0.000000
25%,2.000000,2.000000
50%,3.000000,2.000000
75%,4.000000,2.000000
max,69.000000,5.000000


In [11]:
print(df[df['num_pax']==0].shape)
# Drop the rows if 'num_pax' == 0
df = df[df['num_pax']!=0]
df.shape
# -> Remove the rows since there is no data about pax and the row number is not significant

(196, 34)


(119194, 34)

### Save Cleansed Data as New CSV

In [12]:
df.to_csv('../data/hotel_bookings_v1.csv', index=False)